In [ ]:
import numpy as np
import random

class AntColony:
    def __init__(self, distance_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate):
        self.distance_matrix = distance_matrix
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.num_cities = distance_matrix.shape[0]
        self.pheromone_matrix = np.ones((self.num_cities, self.num_cities))

    def _calculate_probabilities(self, ant, visited):
        pheromone = self.pheromone_matrix[ant, :]
        visibility = 1 / self.distance_matrix[ant, :]
        probabilities = (pheromone ** self.alpha) * (visibility ** self.beta)
        # Convert visited set to array for indexing
        visited = np.array(list(visited))
        probabilities[visited] = 0
        return probabilities / np.sum(probabilities)

    def _update_pheromones(self, all_paths):
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for path, length in all_paths:
            for i in range(len(path) - 1):
                self.pheromone_matrix[path[i], path[i + 1]] += 1 / length

    def solve(self):
        best_path = None
        best_length = float('inf')

        for iteration in range(self.num_iterations):
            all_paths = []
            for ant in range(self.num_ants):
                path = [random.randint(0, self.num_cities - 1)]
                visited = set(path)

                for _ in range(self.num_cities - 1):
                    current_city = path[-1]
                    probabilities = self._calculate_probabilities(current_city, visited)
                    next_city = np.random.choice(range(self.num_cities), p=probabilities)
                    path.append(next_city)
                    visited.add(next_city)

                # Return to starting city
                path.append(path[0])
                length = self._calculate_path_length(path)
                all_paths.append((path, length))

                if length < best_length:
                    best_length = length
                    best_path = path

            self._update_pheromones(all_paths)
            # print(f"Iteration {iteration + 1}, Best Length: {best_length}")

        return best_path, best_length

    def _calculate_path_length(self, path):
        return sum(self.distance_matrix[path[i], path[i + 1]] for i in range(len(path) - 1))


if __name__ == "__main__":
    # Create a distance matrix for 5 cities
    distance_matrix = np.array([[0, 2, 9, 10, 3],
                                 [1, 0, 6, 4, 5],
                                 [15, 7, 0, 8, 9],
                                 [6, 3, 12, 0, 4],
                                 [10, 4, 8, 2, 0]])

    aco = AntColony(distance_matrix, num_ants=10, num_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5)
    best_path, best_length = aco.solve()
    print("Best Path:", best_path)
    print("Best Length:", best_length)

<ipython-input-16-57b8e868a3e4>:17: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / self.distance_matrix[ant, :]


Best Path: [3, 1, 0, 4, 2, 3]
Best Length: 23
